In [32]:
import pandas as pd
import numpy as np
import json

import altair as alt

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
import pyLDAvis
import pyLDAvis.gensim

In [33]:
video_df = pd.read_csv('../../../data/video_transcripts_mix_clean.csv')

In [34]:
video_df.head(3)

,video_id,title,channel_id,channel_title,published_at,view_count,likes,dislikes,comment_count,tags,description,comments,Is_Generated,Transcript_Blob,title_Clean,tags_Clean,description_Clean,Transcript_Blob_Clean
0,--0bCF-iK2E,Jadon Sancho Magical Skills & Goals,UC6UL29enLNe4mqwTfAyeNuw,Bundesliga,2021-07-01 10:00:00,1048888,19515,226,1319,football soccer ftbol alemn Bundesliga season ...,Enjoy the best skills and goals from Jadon San...,"Respect to Dortmund fans,must be sad losing hi...",NaN,NaN,jadon sancho magic skill goal,footbal soccer ftbol alemn bundesliga season 2...,enjoy best skill goal jadon sancho sub httpsre...,NaN
1,--14w5SOEUs,Migos - Avalanche (Official Video),UCGIelM2Dj3zza3xyV3pL3WQ,MigosVEVO,2021-06-10 16:00:00,15352638,359277,7479,18729,Migos Avalanche Quality Control Music/Motown R...,"Watch the the official video for Migos - ""Aval...",Migos just makes me want to live my live to th...,False,[BAND SOUNDS] - HOW WOULD I DESCRIBE\nTHIS AL...,migo avalanch offici video,migo avalanch qualiti control musicmotown reco...,watch offici video migo avalanch streamdownloa...,"['describe', 'album', 'level', 'know', 'saying..."
2,--40TEbZ9Is,Supporting Actress in a Comedy: 73rd Emmys,UClBKH8yZRcM4AsRjDVEdjMg,Television Academy,2021-09-20 01:03:32,925281,11212,401,831,,Hannah Waddingham wins the Emmy for Supporting...,Hannah's energy bursts through any screen. Wel...,True,[Music] hello hi what's happening that was wi...,support actress comedi 73rd emmi,NaN,hannah waddingham win emmi support actress com...,"['happen', 'wild', 'truly', 'insane', 'expect'..."


In [35]:
video_df['clean_combined'] = video_df[['title_Clean','tags_Clean','description_Clean','Transcript_Blob_Clean']].apply( lambda x: ' '.join(x.dropna()), axis=1)

In [36]:
tokenized_vid = [vid.split() for vid in video_df['clean_combined'][:1000].tolist()]

id2word = corpora.Dictionary(tokenized_vid)
corpus = [id2word.doc2bow(tokens) for tokens in tokenized_vid]

In [37]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                            id2word=id2word,
                                            num_topics=20, 
                                            random_state=42,
                                            update_every=1,
                                            chunksize=10000,
                                            passes=10,
                                            alpha='auto', 
                                            eta='auto',
                                            eval_every=None)

In [38]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
12     0.260333 -0.016306       1        1  17.050712
2      0.236270 -0.006643       2        1  15.957723
4      0.239295 -0.007519       3        1  14.357870
17     0.247140  0.009238       4        1  14.072554
8      0.228103 -0.020865       5        1  14.048454
1      0.134181  0.036965       6        1   3.273921
0      0.104623  0.027193       7        1   2.980308
14    -0.015172  0.004397       8        1   2.605191
10    -0.231450 -0.082516       9        1   1.922535
19    -0.102861  0.016850      10        1   1.693419
11    -0.026526 -0.044766      11        1   1.463407
3     -0.118275 -0.017862      12        1   1.451915
18    -0.174717 -0.081623      13        1   1.420371
5     -0.010877 -0.014856      14        1   1.353831
9     -0.170109  0.263553      15        1   1.274527
7     -0.239699 -0.020627      16        1   1.207206
15    -0.098351 -0.028548      17        1   1.199574
13    -0.006785 -0.026981      18        1   1.100104
16    -0.198772 -0.033539      19        1   0.881558
6     -0.056351  0.044456      20        1   0.684820, topic_info=           Term          Freq         Total Category  logprob  loglift
1473         de    612.000000    612.000000  Default  30.0000  30.0000
146      'get',  11038.000000  11038.000000  Default  29.0000  29.0000
556       video    759.000000    759.000000  Default  28.0000  28.0000
150       'go',  12317.000000  12317.000000  Default  27.0000  27.0000
180     'make',   3087.000000   3087.000000  Default  26.0000  26.0000
...         ...           ...           ...      ...      ...      ...
5060      grand      7.571839     54.119362  Topic20  -6.2341   3.0170
2151         tv      7.820939    140.558027  Topic20  -6.2017   2.0950
167     'know',      8.469796   7464.358523  Topic20  -6.1220  -1.7976
716     'work',      8.161853   1406.044678  Topic20  -6.1591  -0.1653
1301  instagram      7.618818    538.332891  Topic20  -6.2279   0.7259

[1490 rows x 6 columns], token_table=       Topic      Freq         Term
term                               
780        1  0.127637  'actually',
780        2  0.265910  'actually',
780        3  0.180819  'actually',
780        4  0.201560  'actually',
780        5  0.175501  'actually',
...      ...       ...          ...
3481      12  0.151530           zu
3481      16  0.707140           zu
43532     15  0.929345        zurdo
10834      9  0.058337           zz
10834     13  0.875060           zz

[5496 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[13, 3, 5, 18, 9, 2, 1, 15, 11, 20, 12, 4, 19, 6, 10, 8, 16, 14, 17, 7])

In [39]:
topic_distributions = [lda_model[doc] for doc in corpus]

df = pd.DataFrame(topic_distributions)

column_names = [f"Topic {i+1}" for i in range(lda_model.num_topics)]
df.columns = column_names



In [40]:
df

,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,Topic 8,Topic 9,Topic 10,Topic 11,Topic 12,Topic 13,Topic 14,Topic 15,Topic 16,Topic 17,Topic 18,Topic 19,Topic 20
0,"(7, 0.9597524)","(18, 0.0365351)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
1,"(3, 0.7175625)","(8, 0.09187125)","(10, 0.17237484)","(19, 0.0126196025)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
2,"(2, 0.03410297)","(4, 0.64670056)","(8, 0.14128475)","(19, 0.16910721)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
3,"(10, 0.989183)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
4,"(0, 0.11790612)","(1, 0.01060921)","(2, 0.014741501)","(14, 0.37874636)","(17, 0.4776053)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,"(1, 0.026147554)","(2, 0.044059943)","(4, 0.21743885)","(5, 0.051318403)","(11, 0.3884625)","(12, 0.019322148)","(16, 0.09906649)","(17, 0.14737153)",None,None,None,None,None,None,None,None,None,None,None,None
996,"(2, 0.18685414)","(3, 0.08516496)","(12, 0.67257184)","(15, 0.052482605)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
997,"(1, 0.024579398)","(12, 0.601392)","(17, 0.37300947)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
998,"(7, 0.60588384)","(8, 0.39113158)",None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None


In [18]:
import sys

print(sys.version)

3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]


In [41]:
fig = px.bar(df, barmode='stack')

# Update the layout for better readability
fig.update_layout(
    xaxis_title='Document',
    yaxis_title='Distribution',
    title='Topic Distributions',
    legend_title='Topics'
)

# Display the chart
fig.show()

c:\users\andre\appdata\local\programs\python\python39\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

c:\users\andre\appdata\local\programs\python\python39\lib\site-packages\plotly\io\_renderers.py:396: DeprecationWarning:

distutils Version classes are deprecated. Use packaging.version instead.

